In [19]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import when
from pyspark.sql.types import StructType, StructField, ShortType, StringType
import datetime
import os

master = "spark://zy-ubuntu:7077"  
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--master {master} --driver-memory 4g --total-executor-cores 8 --executor-memory 8g --packages org.postgresql:postgresql:42.1.1 pyspark-shell'

In [2]:
spark = SparkSession.builder \
    .appName("dim episodes") \
    .getOrCreate()

In [3]:
title_episodes_df = spark.read.csv("title.episode.tsv", sep=r'\t', header=True)
title_episodes_df.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- parentTconst: string (nullable = true)
 |-- seasonNumber: string (nullable = true)
 |-- episodeNumber: string (nullable = true)



In [4]:
title_episodes_df = title_episodes_df.withColumn('seasonNumber', F.col('seasonNumber').cast(ShortType()))
title_episodes_df = title_episodes_df.withColumn('episodeNumber', F.col('episodeNumber').cast(ShortType()))
title_episodes_df = title_episodes_df.drop('tconst')
# title_episodes_df = title_episodes_df.sort('parentTconst', F.col('seasonNumber').desc(), F.col('episodeNumber').desc())
# title_episodes_df.show()

In [5]:
null_df = title_episodes_df.filter(F.col("seasonNumber").isNull() & F.col("episodeNumber").isNull())
null_df_final = null_df.groupBy('parentTconst').count()
null_df_final = null_df_final.withColumn('season', F.lit(None)).withColumnRenamed('count', 'total_episodes').withColumnRenamed('parentTconst', 'tconst')
null_df_final.show()

+---------+--------------+------+
|   tconst|total_episodes|season|
+---------+--------------+------+
|tt2162303|             5|  null|
|tt0094519|             1|  null|
|tt8013750|             3|  null|
|tt0350409|             5|  null|
|tt0418369|             7|  null|
|tt0466509|             2|  null|
|tt0375323|             5|  null|
|tt0295085|             4|  null|
|tt0166438|             2|  null|
|tt0381763|             3|  null|
|tt0485524|            11|  null|
|tt0239181|             1|  null|
|tt0335728|             1|  null|
|tt0168358|           160|  null|
|tt0252778|             1|  null|
|tt0400027|             2|  null|
|tt0085100|            33|  null|
|tt0397147|             2|  null|
|tt0209792|             1|  null|
|tt0787994|           927|  null|
+---------+--------------+------+
only showing top 20 rows



In [6]:
not_null_df = title_episodes_df.filter(F.col("seasonNumber").isNotNull() & F.col("episodeNumber").isNotNull())
not_null_df_final = not_null_df.groupBy('parentTconst', 'seasonNumber').agg(F.max('episodeNumber').alias('episode_count'))
not_null_df_final = not_null_df_final.sort('parentTconst', F.col('seasonNumber').desc(), F.col('episode_count').desc())
not_null_df_final = not_null_df_final.withColumnRenamed('seasonNumber', 'season').withColumnRenamed('parentTconst', 'tconst')
not_null_df_final.show()

+---------+------+-------------+
|   tconst|season|episode_count|
+---------+------+-------------+
|tt0032557|     1|           15|
|tt0038276|     1|            1|
|tt0039120|     1|            1|
|tt0039122|     1|            9|
|tt0039123|    11|           51|
|tt0039123|    10|           53|
|tt0039123|     9|           49|
|tt0039123|     8|           52|
|tt0039123|     7|           56|
|tt0039123|     6|           47|
|tt0039123|     5|           54|
|tt0039123|     4|           50|
|tt0039123|     3|           53|
|tt0039123|     2|           52|
|tt0039123|     1|           70|
|tt0039124|     1|            1|
|tt0039125|     1|           25|
|tt0040021|     2|           30|
|tt0040021|     1|           35|
|tt0040022|     1|            1|
+---------+------+-------------+
only showing top 20 rows



In [7]:
result_df = null_df_final.union(not_null_df_final)
result_df.show()

+---------+--------------+------+
|   tconst|total_episodes|season|
+---------+--------------+------+
|tt2162303|             5|  null|
|tt0094519|             1|  null|
|tt8013750|             3|  null|
|tt0350409|             5|  null|
|tt0418369|             7|  null|
|tt0466509|             2|  null|
|tt0375323|             5|  null|
|tt0295085|             4|  null|
|tt0166438|             2|  null|
|tt0381763|             3|  null|
|tt0485524|            11|  null|
|tt0239181|             1|  null|
|tt0335728|             1|  null|
|tt0168358|           160|  null|
|tt0252778|             1|  null|
|tt0400027|             2|  null|
|tt0085100|            33|  null|
|tt0397147|             2|  null|
|tt0209792|             1|  null|
|tt0787994|           927|  null|
+---------+--------------+------+
only showing top 20 rows



In [8]:
result_df.write.format('jdbc').options(
      url='jdbc:postgresql://localhost:5433/imdb',
      driver='org.postgresql.Driver',
      dbtable='dim_episodes',
      user='admin',
      password='password'
      ).mode('append').save()

In [20]:
# create a df with year, month, day with today's date
now = datetime.datetime.now()

data = [(now.year,now.month,now.day)]

schema = StructType([ \
    StructField("year",ShortType(),True), \
    StructField("month",ShortType(),True), \
    StructField("day",ShortType(),True), \
  ])
 
dl_date_df = spark.createDataFrame(data=data,schema=schema)
dl_date_df.show()

+----+-----+---+
|year|month|day|
+----+-----+---+
|2021|    6| 10|
+----+-----+---+



In [21]:
# insert dataset download date into db
dl_date_df.write.format('jdbc').options(
      url='jdbc:postgresql://localhost:5433/imdb',
      driver='org.postgresql.Driver',
      dbtable='dim_download_date',
      user='admin',
      password='password'
      ).mode('append').save()

In [24]:
title_desc_pg_df = spark.read.format('jdbc').options(
      url='jdbc:postgresql://localhost:5433/imdb',
      driver='org.postgresql.Driver',
      dbtable='dim_title_desc',
      user='admin',
      password='password'
      ).load()

title_desc_pg_df = title_desc_pg_df.withColumnRenamed('id', 'title_desc_id')

In [26]:
fact_titles_df = title_desc_pg_df.select('title_desc_id')

In [27]:
episodes_pg_df = spark.read.format('jdbc').options(
      url='jdbc:postgresql://localhost:5433/imdb',
      driver='org.postgresql.Driver',
      dbtable='dim_episodes',
      user='admin',
      password='password'
      ).load()
      
episodes_pg_df = episodes_pg_df.withColumnRenamed('id', 'episode_id')      

In [28]:
desc_ep_joined_df = episodes_pg_df.join(title_desc_pg_df, ['tconst']).select('episode_id', 'title_desc_id')
fact_titles_df = fact_titles_df.join(desc_ep_joined_df, ['title_desc_id'], how='full')

In [29]:
dl_date_pg_df = spark.read.format('jdbc').options(
      url='jdbc:postgresql://localhost:5433/imdb',
      driver='org.postgresql.Driver',
      dbtable='dim_download_date',
      user='admin',
      password='password'
      ).load()

In [30]:
now = datetime.datetime.now()

df_date_id = dl_date_pg_df.filter(
        (F.col('year') == now.year) & 
        (F.col('month') == now.month) &
        (F.col('day') == now.day))
        
dl_date_id = df_date_id.select('id').collect()[0][0]
fact_titles_df = fact_titles_df.withColumn('download_date_id', F.lit(dl_date_id))

In [31]:
fact_titles_df.write.format('jdbc').options(
      url='jdbc:postgresql://localhost:5433/imdb',
      driver='org.postgresql.Driver',
      dbtable='fact_titles',
      user='admin',
      password='password'
      ).mode('append').save()